In [70]:
import sympy as sp

def commutator(A, B):
    return A * B - B * A

def nested_commutator(A, B, n):
    if n == 1:
        return commutator(A, B)
    else:
        return commutator(A, nested_commutator(A, B, n-1))
def each_level_its_own_subspace(dim):
    """
    Obtain the indices of the subspace to consider.

    Args:
        dim (int): The dimension of the subspace.

    Returns:
        list: A list of lists, each containing a single index of the subspace to consider.
    """
    # Define the indices of the subspace to consider
    subspace_indices = [[i] for i in range(dim)]
    return subspace_indices

def create_subspace_projectors(dim, subspace_indices):
    """
    Create projectors for the given subspaces using SymPy.

    Args:
        dim (int): The total dimension of the Hilbert space.
        subspace_indices (list): A list of lists, each containing the indices of a subspace.

    Returns:
        list: A list of projectors (SymPy matrices) for the given subspaces.
    """
    projectors = []
    
    for subspace in subspace_indices:
        # Initialize the projector matrix P as a zero matrix of size dim x dim
        P = sp.zeros(dim, dim)
        
        # Create the basis states and compute the projector for each index in the subspace
        for idx in subspace:
            # Create a basis state (column vector) with 1 at the index 'idx'
            state = sp.zeros(dim, 1)
            state[idx, 0] = 1
            
            # Compute the rank-1 projector for this basis state: |state><state|
            P += state * state.T  # Outer product in SymPy is `*`
        
        # Add the projector to the list
        projectors.append(P)
    
    return projectors

def swt_subspace(H0, V, subspace_indices):
    """
    Compute the Schrieffer-Wolff transformation on a subspace of the Hamiltonian.

    Args:
        H0 (sp.Matrix): The unperturbed Hamiltonian.
        V (sp.Matrix): The perturbation Hamiltonian.
        subspace_indices (list): The indices of the subspace to consider.

    Returns:
        tuple: The Schrieffer-Wolff transformation operators (S1, S2, S3) and 
               the transformed Hamiltonian components (H1, H2, H3, H4).
    """
    dim = H0.shape[0]
    projectors = create_subspace_projectors(dim, subspace_indices)
    Vd = sp.zeros(dim, dim)
    for P in projectors:
        Vd += P * V * P
    Vod = V - Vd
    # Compute the energy differences in the subspace
    delta = sp.Matrix([[H0[i,i] - H0[j,j] if i != j else 1 for j in range(dim)] for i in range(dim)])

    # Compute the Schrieffer-Wolff transformation components
    H1 = Vd
    S1 = sp.Matrix([[Vod[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])
    H2 = sp.Rational(1, 2) * commutator(S1, Vod)
    
    # S2 = sp.Matrix([[-commutator(Vd, S1)[i,j] / delta[i,j] for j in range(dim)] for i in range(dim)])
    # H3 = sp.Rational(1, 2) * commutator(S2, Vod)
    # S3 = sp.Matrix([[(commutator(S2, Vd)[i,j] + sp.Rational(1, 3) * nested_commutator(S1, Vod, 2)[i,j]) / delta[i,j] for j in range(dim)] for i in range(dim)])
    # H4 = sp.Rational(1, 2) * commutator(S3, Vod) - sp.Rational(1, 24) * nested_commutator(S1, Vod, 3)

    return [S1], [H1, H2,]

In [71]:
from sympy.physics.quantum import TensorProduct
from sympy import symbols, sqrt
nc = 5
nq = 2
def create_annihilation(nc):
    ac = sp.zeros(nc)
    acd = sp.zeros(nc)

    # Fill the matrices using sympy functions
    for i in range(nc-1):
        ac[i, i+1] = sqrt(i+1)
        acd[i+1, i] = sqrt(i+1)
    return ac, acd
aq, aqd = create_annihilation(nq)
ac, acd = create_annihilation(nc)
aq = TensorProduct(aq, sp.eye(nc))
aqd = TensorProduct(aqd, sp.eye(nc))
ac = TensorProduct(sp.eye(nq), ac)
acd = TensorProduct(sp.eye(nq), acd)
sigmaz = TensorProduct( sp.Matrix([[-1,0],[0,1]]), sp.eye(nc))

In [72]:
deltaq, deltac,  = symbols('delta_q delta_c', real=True)
chi = symbols(' chi', real=True)
A = symbols('A', real=True)
g = symbols('g', real=True)
Delta = symbols('Delta', real=True)


In [73]:
aqd

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [74]:
V = A*(aqd + aq) 

ENN = [
    # i*deltac + (l-sp.Rational(1,2))*deltaq + i*2*(l-sp.Rational(1,2))*chi
    i*deltac + l*deltaq + i*l*chi
    for l in range(nq)
    for i in range(nc)
]


In [75]:
H0 = sp.diag(*ENN)

In [76]:
H0

Matrix([
[0,       0,         0,         0,         0,       0,                       0,                           0,                           0,                           0],
[0, delta_c,         0,         0,         0,       0,                       0,                           0,                           0,                           0],
[0,       0, 2*delta_c,         0,         0,       0,                       0,                           0,                           0,                           0],
[0,       0,         0, 3*delta_c,         0,       0,                       0,                           0,                           0,                           0],
[0,       0,         0,         0, 4*delta_c,       0,                       0,                           0,                           0,                           0],
[0,       0,         0,         0,         0, delta_q,                       0,                           0,                           0,              

In [77]:
indices = each_level_its_own_subspace(nq*nc)
H2 = swt_subspace(H0, V, indices)[1][1]
S1 = swt_subspace(H0, V, indices)[0][0]
noise_op = aqd*aq

In [78]:
S1

Matrix([
[        0,                 0,                   0,                   0,                   0, -A/delta_q,                  0,                    0,                    0,                    0],
[        0,                 0,                   0,                   0,                   0,          0, A/(-chi - delta_q),                    0,                    0,                    0],
[        0,                 0,                   0,                   0,                   0,          0,                  0, A/(-2*chi - delta_q),                    0,                    0],
[        0,                 0,                   0,                   0,                   0,          0,                  0,                    0, A/(-3*chi - delta_q),                    0],
[        0,                 0,                   0,                   0,                   0,          0,                  0,                    0,                    0, A/(-4*chi - delta_q)],
[A/delta_q,               

In [79]:
H2

Matrix([
[-A**2/delta_q,                                                     0,                                                         0,                                                         0,                                                         0,            0,                                                    0,                                                        0,                                                        0,                                                        0],
[            0, -A**2/(2*(chi + delta_q)) + A**2/(2*(-chi - delta_q)),                                                         0,                                                         0,                                                         0,            0,                                                    0,                                                        0,                                                        0,                                                        0],
[        

In [80]:
S1

Matrix([
[        0,                 0,                   0,                   0,                   0, -A/delta_q,                  0,                    0,                    0,                    0],
[        0,                 0,                   0,                   0,                   0,          0, A/(-chi - delta_q),                    0,                    0,                    0],
[        0,                 0,                   0,                   0,                   0,          0,                  0, A/(-2*chi - delta_q),                    0,                    0],
[        0,                 0,                   0,                   0,                   0,          0,                  0,                    0, A/(-3*chi - delta_q),                    0],
[        0,                 0,                   0,                   0,                   0,          0,                  0,                    0,                    0, A/(-4*chi - delta_q)],
[A/delta_q,               

In [81]:
def projector(mp,np,m,n):
    P = sp.zeros(nq*nc, nq*nc)
    bra_index = nc*mp + np
    ket_index = nc*m + n
    P[bra_index, ket_index] = 1
    return P

In [82]:
op = sp.zeros(nq*nc, nq*nc)
for i in range(nc):
    op = op + A/(deltaq+i*chi)*(projector(1,i,1,i) - projector(0,i,0,i))
op - commutator(S1,aq)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [83]:
op = sp.zeros(nq*nc, nq*nc)
for i in range(nc):
    op = op + A/(deltaq+i*chi)*(projector(1,i,1,i) - projector(0,i,0,i))
op - commutator(S1,aq)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [84]:
op = sp.zeros(nq*nc, nq*nc)
for i in range(nc-1):
    op = op + A*chi*sp.sqrt(i+1)/(deltaq+i*chi)/(deltaq+(i+1)*chi)*(projector(1,i,0,i+1) - projector(0,i,1,i+1))

In [85]:
(op - commutator(S1,ac)).applyfunc(sp.simplify)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [86]:
op = sp.zeros(nq*nc, nq*nc)
for i in range(nc):
    op = op - A/(deltaq+i*chi)*(projector(0,i,1,i) + projector(1,i,0,i))
(op - commutator(S1,aqd*aq)).applyfunc(sp.simplify)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [87]:
op = sp.zeros(nq*nc, nq*nc)
for i in range(nc-1):
    op = op + A*sp.sqrt(i+1)*(1/(deltaq+(i+1)*chi))*projector(1,i+1,1,i) - A*sp.sqrt(i+1)*(1/(deltaq+(i)*chi))*projector(0,i+1,0,i)
(op - commutator(S1,aq*acd) ).applyfunc(sp.simplify)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [88]:
(commutator(S1,acd)*ac+acd*commutator(S1,ac)).applyfunc(sp.simplify)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [89]:
(commutator(S1,acd)*commutator(S1,ac)).applyfunc(sp.simplify)

Matrix([
[0,                                           0,                                                       0,                                                         0,                                                         0, 0,                                           0,                                                       0,                                                         0,                                                         0],
[0, A**2*chi**2/(delta_q**2*(chi + delta_q)**2),                                                       0,                                                         0,                                                         0, 0,                                           0,                                                       0,                                                         0,                                                         0],
[0,                                           0, 2*A**2*chi**2/((chi + delta_q)**2*(2*chi + del

In [90]:
commutator

<function __main__.commutator(A, B)>

In [91]:
def expand_matrix_terms(matrix, var, point, order):
    expanded_matrix = matrix.applyfunc(lambda term: term.series(var, point, order).removeO())
    return expanded_matrix

In [92]:
commutator(S1,aq)

Matrix([
[-A/delta_q,                  0,                    0,                    0,                    0,         0,                 0,                   0,                   0,                   0],
[         0, -A/(chi + delta_q),                    0,                    0,                    0,         0,                 0,                   0,                   0,                   0],
[         0,                  0, -A/(2*chi + delta_q),                    0,                    0,         0,                 0,                   0,                   0,                   0],
[         0,                  0,                    0, -A/(3*chi + delta_q),                    0,         0,                 0,                   0,                   0,                   0],
[         0,                  0,                    0,                    0, -A/(4*chi + delta_q),         0,                 0,                   0,                   0,                   0],
[         0,              

In [93]:
(expand_matrix_terms(commutator(S1,aq), chi, 0, 2) -( A/deltaq*sigmaz - A*chi/deltaq*sigmaz*acd*ac)).applyfunc(sp.simplify)

Matrix([
[0,                              0,                                0,                                0,                                0, 0,                              0,                                0,                                0,                                0],
[0, A*chi*(1 - delta_q)/delta_q**2,                                0,                                0,                                0, 0,                              0,                                0,                                0,                                0],
[0,                              0, 2*A*chi*(1 - delta_q)/delta_q**2,                                0,                                0, 0,                              0,                                0,                                0,                                0],
[0,                              0,                                0, 3*A*chi*(1 - delta_q)/delta_q**2,                                0, 0,                       

In [94]:
expand_matrix_terms(commutator(S1,ac), chi, 0, 2) - A*chi/deltaq**2*(aqd - aq)*ac

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [95]:

expand_matrix_terms(commutator(S1,aqd)*aq + aqd*commutator(S1,aq), chi, 0, 2) - (-A/deltaq*(aqd+aq) - -A*chi/deltaq**2*(aqd+aq)*acd*ac)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [96]:
expand_matrix_terms(commutator(S1,aqd)*commutator(S1,aq), chi, 0, 2)

Matrix([
[A**2/delta_q**2,                                        0,                                        0,                                        0,                                        0,               0,                                        0,                                        0,                                        0,                                        0],
[              0, -2*A**2*chi/delta_q**3 + A**2/delta_q**2,                                        0,                                        0,                                        0,               0,                                        0,                                        0,                                        0,                                        0],
[              0,                                        0, -4*A**2*chi/delta_q**3 + A**2/delta_q**2,                                        0,                                        0,               0,                                        0,   

In [97]:
expand_matrix_terms(commutator(S1,aq*acd), chi, 0, 2)

Matrix([
[         0,                                            0,                                              0,                                0, 0,                             0,                                               0,                                               0,                                 0, 0],
[-A/delta_q,                                            0,                                              0,                                0, 0,                             0,                                               0,                                               0,                                 0, 0],
[         0, sqrt(2)*A*chi/delta_q**2 - sqrt(2)*A/delta_q,                                              0,                                0, 0,                             0,                                               0,                                               0,                                 0, 0],
[         0,                                           

In [98]:
acd*ac*acd - acd 

Matrix([
[0,       0,         0, 0, 0, 0,       0,         0, 0, 0],
[0,       0,         0, 0, 0, 0,       0,         0, 0, 0],
[0, sqrt(2),         0, 0, 0, 0,       0,         0, 0, 0],
[0,       0, 2*sqrt(3), 0, 0, 0,       0,         0, 0, 0],
[0,       0,         0, 6, 0, 0,       0,         0, 0, 0],
[0,       0,         0, 0, 0, 0,       0,         0, 0, 0],
[0,       0,         0, 0, 0, 0,       0,         0, 0, 0],
[0,       0,         0, 0, 0, 0, sqrt(2),         0, 0, 0],
[0,       0,         0, 0, 0, 0,       0, 2*sqrt(3), 0, 0],
[0,       0,         0, 0, 0, 0,       0,         0, 6, 0]])

In [102]:
(commutator(S1,sigmaz@ac))

Matrix([
[0,                              0,                                                        0,                                                          0,                                              0, 0, A/(-chi - delta_q) - A/delta_q,                                                         0,                                                           0,                                               0],
[0,                              0,                                                        0,                                                          0,                                              0, 0,                              0, sqrt(2)*A/(-chi - delta_q) + sqrt(2)*A/(-2*chi - delta_q),                                                           0,                                               0],
[0,                              0,                                                        0,                                                          0,                        

In [103]:
expand_matrix_terms((commutator(S1,sigmaz@ac)), chi, 0, 2) 

Matrix([
[0,                              0,                                                0,                                                0,                                 0, 0, A*chi/delta_q**2 - 2*A/delta_q,                                                0,                                                0,                                 0],
[0,                              0,                                                0,                                                0,                                 0, 0,                              0, 3*sqrt(2)*A*chi/delta_q**2 - 2*sqrt(2)*A/delta_q,                                                0,                                 0],
[0,                              0,                                                0,                                                0,                                 0, 0,                              0,                                                0, 5*sqrt(3)*A*chi/delta_q**2 - 2*sqrt(3)*A/delta_q,  

In [106]:
(aq+aqd)*ac

Matrix([
[0, 0,       0,       0, 0, 0, 1,       0,       0, 0],
[0, 0,       0,       0, 0, 0, 0, sqrt(2),       0, 0],
[0, 0,       0,       0, 0, 0, 0,       0, sqrt(3), 0],
[0, 0,       0,       0, 0, 0, 0,       0,       0, 2],
[0, 0,       0,       0, 0, 0, 0,       0,       0, 0],
[0, 1,       0,       0, 0, 0, 0,       0,       0, 0],
[0, 0, sqrt(2),       0, 0, 0, 0,       0,       0, 0],
[0, 0,       0, sqrt(3), 0, 0, 0,       0,       0, 0],
[0, 0,       0,       0, 2, 0, 0,       0,       0, 0],
[0, 0,       0,       0, 0, 0, 0,       0,       0, 0]])

In [107]:
expand_matrix_terms((commutator(S1,aqd@ac)), chi, 0, 2) 

Matrix([
[0, -A/delta_q,                                            0,                                              0,                                0, 0,                             0,                                               0,                                               0,                                 0],
[0,          0, sqrt(2)*A*chi/delta_q**2 - sqrt(2)*A/delta_q,                                              0,                                0, 0,                             0,                                               0,                                               0,                                 0],
[0,          0,                                            0, 2*sqrt(3)*A*chi/delta_q**2 - sqrt(3)*A/delta_q,                                0, 0,                             0,                                               0,                                               0,                                 0],
[0,          0,                                        

In [109]:
sigmaz*ac

Matrix([
[0, -1,        0,        0,  0, 0, 0,       0,       0, 0],
[0,  0, -sqrt(2),        0,  0, 0, 0,       0,       0, 0],
[0,  0,        0, -sqrt(3),  0, 0, 0,       0,       0, 0],
[0,  0,        0,        0, -2, 0, 0,       0,       0, 0],
[0,  0,        0,        0,  0, 0, 0,       0,       0, 0],
[0,  0,        0,        0,  0, 0, 1,       0,       0, 0],
[0,  0,        0,        0,  0, 0, 0, sqrt(2),       0, 0],
[0,  0,        0,        0,  0, 0, 0,       0, sqrt(3), 0],
[0,  0,        0,        0,  0, 0, 0,       0,       0, 2],
[0,  0,        0,        0,  0, 0, 0,       0,       0, 0]])